In [1]:
%load_ext autoreload
%autoreload 1
%aimport pygenesig.pybioqc
%aimport pygenesig.tools
%aimport pygenesig.file_formats
%matplotlib inline


from pygenesig.pybioqc import * 
from pygenesig.bioqc import * 
from pygenesig.tools import * 
from pygenesig.file_formats import * 
from scipy.stats import mannwhitneyu
import pandas as pd
from pylab import * 
import seaborn as sns

In [2]:
gtex_sig = load_gmt("../pygenesig-example/results/gtex_v6_gini_0.8_3/signatures.gmt")

In [4]:
exprs = read_expr("../pygenesig-example/data_processed/v6/exprs.npy")
target = read_target("../pygenesig-example/data_processed/v6/target.csv")
rosetta = read_rosetta("../pygenesig-example/data_processed/v6/rosetta.csv", inverse=True)

In [5]:
gtex_sig_gtex = translate_signatures(gtex_sig, rosetta)

In [29]:
rel_ranks = {}
for tissue, inds in gtex_sig_gtex.items():
    rel_ranks[tissue] = np.median(pd.DataFrame(exprs[:, target == tissue]).rank(axis=0, ascending=False).values[inds, :].flatten()) / exprs.shape[0]

In [31]:
rel_ranks_df = pd.DataFrame([x for x in rel_ranks.items()])

In [35]:
rel_ranks_df.to_csv("results/rel_ranks_gtex_ngs.tsv", sep="\t", header=None, index=None)

## The same for mouse microarray

In [37]:
exprs = read_expr("../pygenesig-example/data_processed/mouseGNF/exprs.npy")
target = read_target("../pygenesig-example/data_processed/mouseGNF/target.csv")
rosetta = read_rosetta("../pygenesig-example/data_processed/mouseGNF/rosetta.csv", inverse=True)

In [58]:
gtex_sig_mousegnf = translate_signatures(gtex_sig, rosetta, ignore_missing=True)

In [59]:
tissue_map = pd.read_csv("../pygenesig-example/manual_annotation/mouseGNF_map_tissues.csv", sep=";", header=None)
tissue_map = dict(zip(tissue_map[1], tissue_map[0]))

In [60]:
gtex_sig_mousegnf = {
    tissue: ind for tissue, ind in gtex_sig_mousegnf.items() if tissue in tissue_map
}

In [62]:
rel_ranks = {}
for tissue, inds in gtex_sig_mousegnf.items():
    rel_ranks[tissue] = np.median(pd.DataFrame(exprs[:, target == tissue_map[tissue]]).rank(axis=0, ascending=False).values[inds, :].flatten()) / exprs.shape[0]

In [64]:
rel_ranks

{'Adrenal_Gland': 0.6530818244905684,
 'Bladder': 0.37697937038672641,
 'Brain_Amygdala': 0.37950287048135767,
 'Brain_Cerebellum': 0.18181818181818182,
 'Brain_Hippocampus': 0.2356633650873762,
 'Brain_Hypothalamus': 0.15869661220112297,
 'Liver': 0.046211595482934832,
 'Lung': 0.53088133240804991,
 'Muscle_Skeletal': 0.053372027001451015,
 'Ovary': 0.63563813008642989,
 'Pancreas': 0.17781212541795471,
 'Pituitary': 0.50009463125354869,
 'Prostate': 0.39839757743990917,
 'Spleen': 0.38612705822976467,
 'Stomach': 0.15320799949529998,
 'Testis': 0.13033877988770426,
 'Uterus': 0.33225033120938741}

In [65]:
rel_ranks_df = pd.DataFrame([x for x in rel_ranks.items()])
rel_ranks_df.to_csv("results/rel_ranks_gtex_mouse_affy.tsv", sep="\t", header=None, index=None)